### Solving Poisson PDE in 2D using FEM + AMR


Starting with uploading nessesary libraries

In [2]:
using Gridap, Gridap.Geometry, Gridap.Adaptivity
using DataStructures

Define analytical solution

In [3]:
# Analytical solution: u_exact(x,y) = exp(-10 * x^2 -10 * y^2)
u_exact(x) = exp(-10 * x[1]^2 -10 * x[2]^2)


u_exact (generic function with 1 method)

Define domain and boundary conditions


In [4]:
#Domain definition
domain = (-2, 2, -2, 2)
n_cells = (10, 10)  # initial mesh resolution
model = simplexify(CartesianDiscreteModel(domain, n_cells))

UnstructuredDiscreteModel()

Define L$^2$ norm for error estimation

In [5]:
l2_norm(he,xh,dΩ) = ∫(he*(xh*xh))*dΩ
l2_norm(xh,dΩ) = ∫(xh*xh)*dΩ


l2_norm (generic function with 2 methods)

AMR step function. Basically, this block represent one full iteration of adaptive mesh refinement process. Here we solve our equation on the given mesh, then according to DoflerMarking and residual error value we refine the mesh and return new model(mesh), approximated solution, residual error and absolute error values.

In [6]:
#AMR step function

function amr_step(model,u_exact;order=2)
  #"Create FE spaces with Dirichlet boundary conditions on all boundaries"
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = TestFESpace(model,reffe;dirichlet_tags=["boundary"])
  U = TrialFESpace(V,u_exact)

  #"Setup integration measures"
  Ω = Triangulation(model)
  Γ = Boundary(model)
  Λ = Skeleton(model)

  dΩ = Measure(Ω,4*order)
  dΓ = Measure(Γ,2*order)
  dΛ = Measure(Λ,2*order)

  "Compute cell sizes for error estimation"
  hK = CellField(sqrt.(collect(get_array(∫(1)dΩ))),Ω)

  "Get normal vectors for boundary and interface terms"
  nΓ = get_normal_vector(Γ)
  nΛ = get_normal_vector(Λ)

  "Define the weak form"
  ∇u(x)  = ∇(u_exact)(x)
  f(x)   = 40*(10(x[1]^2 + x[2]^2) - 1)*exp(-10 * x[1]^2 -10 * x[2]^2)
  a(u,v) = (-1)*∫(∇(u)⋅∇(v))dΩ
  l(v)   = ∫(f*v)dΩ

  "Define the residual error estimator
  It includes volume residual, boundary jump, and interface jump terms"
  ηh(u)  = l2_norm(hK*(f + Δ(u)),dΩ) +           # Volume residual
           l2_norm(hK*(∇(u) - ∇u)⋅nΓ,dΓ) +       # Boundary jump
           l2_norm(jump(hK*∇(u)⋅nΛ),dΛ)          # Interface jump

  "Solve the FE problem"
  op = AffineFEOperator(a,l,U,V)
  uh = solve(op)

  "Compute error indicators"
  η = estimate(ηh,uh)

  "Mark cells for refinement using Dörfler marking
  This strategy marks cells containing a fixed fraction (0.9) of the total error"
  m = DorflerMarking(0.9)
  I = Adaptivity.mark(m,η)

  "Refine the mesh using newest vertex bisection"
  method = Adaptivity.NVBRefinement(model)
  amodel = refine(method,model;cells_to_refine=I)
  fmodel = Adaptivity.get_model(amodel)

  "Compute the global error for convergence testing"
  error = sum(l2_norm(uh - u_exact,dΩ))
  return fmodel, uh, η, I, error
end

amr_step (generic function with 1 method)

Now set maximal iterations value and order that will affect basic functions(Lagrange polynom degree) and measures.

In [7]:
nsteps = 5
order = 2


2

Here is the iterative algorithm that calls our function above and iteratively refines our map

In [8]:
last_error = Inf
for i in 1:nsteps
  fmodel, uh, η, I, error = amr_step(model,u_exact;order)

  is_refined = map(i -> ifelse(i ∈ I, 1, -1), 1:num_cells(model))

  Ω = Triangulation(model)
  writevtk(
    Ω,"model_$(i-1)",append=false,
    cellfields = [
      "uh" => uh,                    # Computed solution
     "η" => CellField(η,Ω),        # Error indicators
      "is_refined" => CellField(is_refined,Ω),  # Refinement markers
      "u_exact" => CellField(u_exact,Ω),       # Exact solution
    ],
  )

  println("Error: $error, Error η: $(sum(η))")
  last_error = error
  model = fmodel
end

Error: 0.0009620832295332464, Error η: 16.646355144656347
Error: 0.0002139425978642257, Error η: 12.35201692761035
Error: 4.09737794781507e-5, Error η: 9.247794030703847
Error: 1.498397484009759e-5, Error η: 5.558463038973415
Error: 1.9991985829972445e-6, Error η: 2.9232022215267826
